<a href="https://colab.research.google.com/github/YevheniiKshanovskyi/SFLteam/blob/main/lab_football/SFL_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Насправді трохи шкода, що ми не презентували цей проєкт на парі, бо ми над ним дуже старалися та зробили чималий речерч, один з наших тіммейтів виявився знавцем футболу та сильно допоміг нам у формуванні завдань та генеруванням нових даних, яких не було в датасеті. Завдяки цьому робота над ним була дуже цікавою) Завдяки новим показникам, такими як Key passes, ми змогли вирахувати інші цікаві коефіцієнти, які ми заресерчили.

In [ ]:
# Run firstly to import required libraries
import pandas as pd

In [ ]:
# Run to read the dataset
data = pd.read_csv('https://raw.githubusercontent.com/YevheniiKshanovskyi/SFLteam/main/lab_football/Euro_2012_stats_TEAM%20(1).csv')
data

,Team,Goals,Shots on target,Shots off target,Shooting Accuracy,% Goals-to-shots,Total shots (inc. Blocked),Hit Woodwork,Penalty goals,Penalties not scored,...,Saves made,Saves-to-shots ratio,Fouls Won,Fouls Conceded,Offsides,Yellow Cards,Red Cards,Subs on,Subs off,Players Used
0,Croatia,4,13,12,51.9%,16.0%,32,0,0,0,...,13,81.3%,41,62,2,9,0,9,9,16
1,Czech Republic,4,13,18,41.9%,12.9%,39,0,0,0,...,9,60.1%,53,73,8,7,0,11,11,19
2,Denmark,4,10,10,50.0%,20.0%,27,1,0,0,...,10,66.7%,25,38,8,4,0,7,7,15
3,England,5,11,18,50.0%,17.2%,40,0,0,0,...,22,88.1%,43,45,6,5,0,11,11,16
4,France,3,22,24,37.9%,6.5%,65,1,0,0,...,6,54.6%,36,51,5,6,0,11,11,19
5,Germany,10,32,32,47.8%,15.6%,80,2,1,0,...,10,62.6%,63,49,12,4,0,15,15,17
6,Greece,5,8,18,30.7%,19.2%,32,1,1,1,...,13,65.1%,67,48,12,9,1,12,12,20
7,Italy,6,34,45,43.0%,7.5%,110,2,0,0,...,20,74.1%,101,89,16,16,0,18,18,19
8,Netherlands,2,12,36,25.0%,4.1%,60,2,0,0,...,12,70.6%,35,30,3,5,0,7,7,15
9,Poland,2,15,23,39.4%,5.2%,48,0,0,0,...,6,66.7%,48,56,3,7,1,7,7,17


1. Find ratio of team's and opponent's blocks

In [ ]:
data['total shots'] =  (data['Shots on target'] + data['Shots off target'])
data['blocked own shots'] = (data['Total shots (inc. Blocked)'] - data['total shots'])
data['Block rate %'] = (round((data['Blocks'] / data['blocked own shots'])*100, 2))
block_ratings = data[['Team', 'Block rate %', 'Goals', 'Goals conceded']]
block_ratings.sort_values(by='Block rate %', ascending=False).head()

,Team,Block rate %,Goals,Goals conceded
14,Sweden,400.00,5,5
6,Greece,383.33,5,7
3,England,263.64,5,3
11,Republic of Ireland,255.56,1,9
0,Croatia,142.86,4,3


2. The percentage of hits from all touches

In [ ]:
touch = data.groupby('Team')['Touches', 'Goals'].sum()
for item in touch:
  print(round((touch['Goals']/touch['Touches'])*100, 2), '%')
  break

Team
Croatia                0.23
Czech Republic         0.17
Denmark                0.21
England                0.20
France                 0.10
Germany                0.27
Greece                 0.25
Italy                  0.14
Netherlands            0.09
Poland                 0.12
Portugal               0.20
Republic of Ireland    0.07
Russia                 0.22
Spain                  0.21
Sweden                 0.28
Ukraine                0.11
dtype: float64 %


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


3. Calculate key passes for teams

In [ ]:
data['Key Passes Rate'] =  data['Passes'] / (data['Shots on target'] - data['Goals'] + data['Penalty goals'] + data['Penalties not scored'])
data['Key Passes'] = round((data['Passes'] /  data['Key Passes Rate']) * (round(data['% Goals-to-shots'].apply(lambda x: float(x.split()[0].replace('%', ''))).mean(),-1)),1)
Key_passes = data[['Team', 'Goals', 'Key Passes']].sort_values(by='Key Passes', ascending = False)
Key_passes.head()


,Team,Goals,Key Passes
13,Spain,12,310.0
7,Italy,6,280.0
5,Germany,10,230.0
4,France,3,190.0
10,Portugal,6,160.0


4. Calculate percentage of unusual goals, like penalty or headed goals

In [ ]:
unusuals_goal = data.loc[:, ['Team', 'Goals', 'Headed goals', 'Penalty goals']]
unusuals_goal['Percentage of Headed goals'] = round((unusuals_goal['Headed goals']/unusuals_goal['Goals']*100), 2)
unusuals_goal['Percentage of Penalty goals'] = round((unusuals_goal['Penalty goals']/unusuals_goal['Goals']*100), 2)
unusuals_goal[['Team', 'Headed goals', 'Percentage of Headed goals', 'Penalty goals', 'Percentage of Penalty goals']]


,Team,Headed goals,Percentage of Headed goals,Penalty goals,Percentage of Penalty goals
0,Croatia,2,50.00,0,0.00
1,Czech Republic,0,0.00,0,0.00
2,Denmark,3,75.00,0,0.00
3,England,3,60.00,0,0.00
4,France,0,0.00,0,0.00
5,Germany,2,20.00,1,10.00
6,Greece,0,0.00,1,20.00
7,Italy,2,33.33,0,0.00
8,Netherlands,0,0.00,0,0.00
9,Poland,1,50.00,0,0.00


5. Find the most effective relative to passes team

In [ ]:
data['Touches without pass'] = data['Touches'] - (data['Passes']*1.2 + data['Saves made']*2)
data['Touches per dribble'] = data['Touches without pass'] / data['Dribbles']
data['Team Work'] = round((data['Passes completed'] / data['Touches per dribble']) / 100, 2)
Dribble_stats = data[['Team','Goals', 'Touches per dribble', 'Passes completed', 'Team Work']].sort_values(by='Team Work', ascending=False)
Dribble_stats.head(5)

,Team,Goals,Touches per dribble,Passes completed,Team Work
13,Spain,12,3.533962,3820,10.81
5,Germany,10,6.870000,2427,3.53
4,France,3,5.497368,1803,3.28
7,Italy,6,9.384000,2531,2.70
8,Netherlands,2,5.546939,1381,2.49


6. Calculate Accurate through balls

In [ ]:
data['Accurate Through Balls'] = round((data['Key Passes'] * 0.235) / data['Crosses'], 2)
data[['Team', 'Accurate Through Balls']].head()

,Team,Accurate Through Balls
0,Croatia,0.35
1,Czech Republic,0.46
2,Denmark,0.33
3,England,0.24
4,France,0.81


7. Calculate the defensive parameter for teams

In [ ]:
data['Def'] = data['Tackles'] + data['Interceptions']
data[['Team', 'Def']].head()

,Team,Def
0,Croatia,105
1,Czech Republic,99
2,Denmark,99
3,England,158
4,France,129


8. Calculate percentage of attacking passes 

In [ ]:
data['Key Passes Rate'] =  data['Passes'] / (data['Shots on target'] - data['Goals'] + data['Penalty goals'] + data['Penalties not scored'])
data['Key Passes'] = round((data['Passes'] /  data['Key Passes Rate']) * (round(data['% Goals-to-shots'].apply(lambda x: float(x.split()[0].replace('%', ''))).mean(),1)),1)
data['Accurate Through Balls'] = (data['Key Passes'] * 0.235) / data['Crosses']
data['%AP'] = round((data['Key Passes'] + data['Accurate Through Balls']) / data['Passes'], 2)
data[['Team', '%AP']].head()

,Team,%AP
0,Croatia,0.10
1,Czech Republic,0.07
2,Denmark,0.05
3,England,0.05
4,France,0.11


9. Calculate the attacking parameter for teams

In [ ]:
data['Key Passes Rate'] =  data['Passes'] / (data['Shots on target'] - data['Goals'] + data['Penalty goals'] + data['Penalties not scored'])
data['Key Passes'] = round((data['Passes'] /  data['Key Passes Rate']) * (round(data['% Goals-to-shots'].apply(lambda x: float(x.split()[0].replace('%', ''))).mean(),1)),1)
Key_passes = data[['Team', 'Goals', 'Key Passes']]
data['Accurate Through Balls'] = (data['Key Passes'] * 0.235) / data['Crosses']
data['ATT'] = round(data['Key Passes'] + data['Accurate Through Balls'], 2)
data[['Team', 'ATT']].head()

,Team,ATT
0,Croatia,105.71
1,Czech Republic,105.84
2,Denmark,70.58
3,England,70.48
4,France,223.25


10. Calculate the index of the attacking implementation

In [ ]:
data['Key Passes Rate'] =  data['Passes'] / (data['Shots on target'] - data['Goals'] + data['Penalty goals'] + data['Penalties not scored'])
data['Key Passes'] = round((data['Passes'] /  data['Key Passes Rate']) * (round(data['% Goals-to-shots'].apply(lambda x: float(x.split()[0].replace('%', ''))).mean(),1)),1)
Key_passes = data[['Team', 'Goals', 'Key Passes']]
data['Accurate Through Balls'] = (data['Key Passes'] * 0.235) / data['Crosses']
data['ATT'] = data['Key Passes'] + data['Accurate Through Balls']

data['IAR'] = round(data['ATT'] * (data['% Goals-to-shots'].apply(lambda x: float(x.split()[0].replace('%', ''))) / 100), 2)
aia = data[['Team', 'Goals', 'IAR']]
aia.head()

,Team,Goals,IAR
0,Croatia,4,16.91
1,Czech Republic,4,13.65
2,Denmark,4,14.12
3,England,5,12.12
4,France,3,14.51
